In [ ]:
from torchvision.models import resnet50 as _resnet50, ResNet50_Weights
from PIL import Image
import torch
from torchvision import transforms

# creating a function for model inferencing  :
def resnet50(**kwargs):
    """ # This docstring shows up in hub.help()
    Resnet18 model
    pretrained (bool): kwargs, load pretrained weights into the model
    """
    # Call the model, load pretrained weights
    model = _resnet50(weights=ResNet50_Weights.IMAGENET1K_V1, **kwargs)
    return model
convert_tensor = transforms.ToTensor()
from torchvision.models import resnet50, ResNet50_Weights
# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT

# Inference
model = resnet50()

Reading the video and saving its frames. The video used is in the link below:
https://www.youtube.com/shorts/ara-UQzTkk4


In [ ]:
import cv2

# Reading the video
vidcap = cv2.VideoCapture('/path/to/video.mp4')
# reading each frame as an image
success,image = vidcap.read()
# initialising the count of frames
count = 0
# The frame must be read successfully before we can proceed with saving it
while success:
  # save frame as JPEG file 
  cv2.imwrite("frame%d.jpg" % count, image)          
  success,image = vidcap.read()
  #print('Read a new frame: ', success)
  count += 1

Inference ResNet50 on the video and saving the results in the directory test

In [ ]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import matplotlib.pyplot as plt
from PIL import Image
import cv2

# Step 1: Initialize model with the best available weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.eval()
# creating a directory named test to store the results 
!mkdir test 

for i in range (0,count):
  # reading the images one by one
  img = read_image(f'/content/frame{i}.jpg')
  # using the open cv read to plot the images
  plot_image = cv2.imread(f'/content/frame{i}.jpg')
  #Image.open(f'/content/frame{i}.jpg')
  #ploting the figure
  plt.figure()
  plt.imshow(plot_image)
  # Step 2: Initialize the inference transforms
  preprocess = weights.transforms()
  # Step 3: Apply inference preprocessing transforms
  batch = preprocess(img).unsqueeze(0)

  # Step 4: Use the model and print the predicted category
  prediction = model(batch).squeeze(0).softmax(0)
  class_id = prediction.argmax().item()
  score = prediction[class_id].item()
  category_name = weights.meta["categories"][class_id]
  # Puting text on top of the image
  cv2.putText(img=plot_image, text=f"{category_name}: {100 * score:.1f}%",org=(10, 50), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=0.8, color=(0,0,255), thickness=2)
  # saving the images in the test directory
  cv2.imwrite(f"/content/test/frame_estimation{i}.jpg", plot_image)
  plt.title(f"{category_name}: {100 * score:.1f}%")


In [ ]:
# zipping the test file that contains images 
!zip -r /content/test /content/test
# After zipping the file download it